# Download Dependencies

In [ ]:
!pip install tensorflow opencv-python numpy

^C


# Imports

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
import glob

# U-Net Model

In [ ]:
def create_unet_model(input_shape):
    inputs = Input(input_shape)

    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    # Add more convolution and pooling layers here as needed
    # Example:
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    # ...

    # Decoder
    up3 = UpSampling2D(size=(2, 2))(conv3)  # Ensure you define 'conv3' here
    concat3 = concatenate([conv2, up3], axis=-1)
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(concat3)

    # Add more upsampling and concatenation layers as needed
    # Example:
    up4 = UpSampling2D(size=(2, 2))(conv4)
    concat4 = concatenate([conv1, up4], axis=-1)
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(concat4)

    # ...

    outputs = Conv2D(1, 1, activation='sigmoid')(conv5)  # Ensure you define 'conv5' here

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Define the input shape
input_shape = (256, 256, 3)  # Adjust the input shape according to your images

# Create the U-Net model
model = create_unet_model(input_shape)

# Define the path to your image directory
image_dir = r'D:\Users\Nadeem\Desktop\BSDS\FYP\Data\Acne\Classification\JPEGImages'

# Load and preprocess images from the directory
def load_and_preprocess_images(image_dir, input_shape):
    image_paths = glob.glob(os.path.join(image_dir, '*.jpg'))
    images = []
    for image_path in image_paths:
        img = cv2.imread(image_path)
        img = cv2.resize(img, (input_shape[0], input_shape[1]))
        img = img / 255.0  # Normalize to [0, 1]
        images.append(img)
    return np.array(images)

# Load the images
images = load_and_preprocess_images(image_dir, input_shape)

# Predict the segmentation masks for the loaded images
masks = model.predict(images)

# Now you have the segmentation masks for the images in the 'masks' variable

46/46 [==============================] - 405s 9s/step


In [ ]:
new_image_path = 'acne-cystic-46.jpg'
new_image = cv2.imread(new_image_path)
input_shape = (256, 256)  # Specify the desired input shape
new_image = cv2.resize(new_image, (input_shape[0], input_shape[1]))
new_image_normalized = new_image / 255.0  # Normalize to [0, 1]

# Predict the segmentation mask for the new image
segmentation_mask = model.predict(np.expand_dims(new_image_normalized, axis=0))[0]

# Threshold the mask to get binary segmentation
threshold = 0.5  # Adjust as needed
binary_mask = (segmentation_mask > threshold).astype(np.uint8)

# Create a green mask
green_mask = np.zeros_like(new_image)
green_mask[:,:,1] = 255  # Set the green channel to 255 (green color)

# Apply the binary mask to the green mask
segmented_image = cv2.bitwise_and(green_mask, green_mask, mask=binary_mask)

# Overlay the segmented mask on the original image
overlay = cv2.addWeighted(new_image, 0.7, segmented_image, 0.3, 0)

# Display the original image and segmented result side by side
cv2.imshow('Original Image', new_image)
cv2.imshow('Segmented Image', overlay)
cv2.waitKey(0)
cv2.destroyAllWindows()

1/1 [==============================] - 0s 332ms/step


# Seg-Net Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization
from tensorflow.keras.models import Model

def create_segnet_model(input_shape, num_classes):
    # Define the input layer
    inputs = Input(shape=input_shape)

    # Encoder
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Decoder
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)

    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)

    # Output layer
    outputs = Conv2D(num_classes, (1, 1), activation='softmax', padding='valid')(x)

    # Create the model
    model = Model(inputs, outputs)

    return model

input_shape = (256, 256, 3)  # Adjust the input shape according to your images
num_classes = 21  # Number of segmentation classes (adjust as needed)

segnet_model = create_segnet_model(input_shape, num_classes)

segnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

segnet_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 256, 256, 64)      1792      
                                                                 
 batch_normalization (Batch  (None, 256, 256, 64)      256       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 64)      36928     
                                                                 
 batch_normalization_1 (Bat  (None, 256, 256, 64)      256       
 chNormalization)                                                
                                                                 
 max_pooling2d (MaxPooling2  (None, 128, 128, 64)      0     

In [ ]:
import cv2
import numpy as np

# Load the image
image_path = 'acne-cystic-46.jpg'
image = cv2.imread(image_path)
image = cv2.resize(image, (256, 256))  # Resize to match the input size of the model
image_normalized = image / 255.0  # Normalize to [0, 1]

# Perform inference to obtain the segmentation mask
segmentation_mask = segnet_model.predict(np.expand_dims(image_normalized, axis=0))[0]

# Convert the mask to a colored segmentation map
colored_mask = np.zeros((256, 256, 3), dtype=np.uint8)

# Define colors for each class (adjust as needed)
class_colors = {
    0: [0, 0, 0],    # Background (black)
    1: [0, 255, 0],  # Class 1 (green)
    2: [0, 0, 255],  # Class 2 (red)
    # Add more colors for additional classes as necessary
}

# Assign colors to each pixel based on the class with the highest probability
for class_index, color in class_colors.items():
    mask = segmentation_mask[:, :, class_index]
    colored_mask[mask > 0.5] = color

# Overlay the colored mask on the original image
segmented_image = cv2.addWeighted(image, 0.7, colored_mask, 0.3, 0)

# Display the original image and segmented result
cv2.imshow('Original Image', image)
cv2.imshow('Segmented Image', segmented_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

1/1 [==============================] - 0s 273ms/step


# Classification Model

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
data_dir = 'D:\\Users\\Data\\Main_data'
image_size = (224, 224)
batch_size = 32

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Split 20% of the data for validation
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 5946 images belonging to 9 classes.
Found 1482 images belonging to 9 classes.


In [ ]:
base_model = MobileNetV2(input_shape=(image_size[0], image_size[1], 3), include_top=False, weights='imagenet')

9406464/9406464 [==============================] - 6s 1us/step


In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(9, activation='softmax')(x)  # 9 classes for your skin diseases

model = Model(inputs=base_model.input, outputs=output)

In [ ]:
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
epochs = 20
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs,
    verbose=1
)

Epoch 1/20
185/185 [==============================] - 1057s 6s/step - loss: 1.0278 - accuracy: 0.5960 - val_loss: 15.1489 - val_accuracy: 0.0890
Epoch 2/20
185/185 [==============================] - 908s 5s/step - loss: 0.8921 - accuracy: 0.6561 - val_loss: 5.7942 - val_accuracy: 0.2887
Epoch 3/20
185/185 [==============================] - 924s 5s/step - loss: 0.8043 - accuracy: 0.6923 - val_loss: 17.5522 - val_accuracy: 0.1827
Epoch 4/20
185/185 [==============================] - 918s 5s/step - loss: 0.7134 - accuracy: 0.7208 - val_loss: 10.0157 - val_accuracy: 0.2276
Epoch 5/20
185/185 [==============================] - 933s 5s/step - loss: 0.7165 - accuracy: 0.7274 - val_loss: 17.9518 - val_accuracy: 0.2276
Epoch 6/20
185/185 [==============================] - 924s 5s/step - loss: 0.6628 - accuracy: 0.7454 - val_loss: 4.6493 - val_accuracy: 0.2908
Epoch 7/20
185/185 [==============================] - 864s 5s/step - loss: 0.6137 - accuracy: 0.7587 - val_loss: 4.1836 - val_accuracy: 0

In [ ]:
model.evaluate(validation_generator)

47/47 [==============================] - 72s 2s/step - loss: 6.6139 - accuracy: 0.2901


[6.613879680633545, 0.29014843702316284]

# Testing of Classification Model

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model('skin_disease_classification_model.h5')

In [ ]:
import cv2
import numpy as np

new_image = cv2.imread('monkeypox1.png')
new_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB)
new_image = cv2.resize(new_image, (224, 224))
new_image = np.expand_dims(new_image, axis=0) / 255.0  # Normalize the image

predictions = model.predict(new_image)

class_labels = ['Acne', 'Atopic Dermatitis', 'Chickenpox', 'Eczema', 'Measles', 'Monkeypox', 'Normal', 'Psoriais', 'Urticaria Hives']  # Replace with your class labels
predicted_class_index = np.argmax(predictions)
predicted_class = class_labels[predicted_class_index]

print(f'Predicted class: {predicted_class}')

1/1 [==============================] - 0s 83ms/step
Predicted class: Urticaria Hives
